In [17]:
# librerias
import h5py as h5
from pyhdf.SD import SD, SDC
import tables
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from numpy import meshgrid
from scipy.interpolate import griddata as gd
import os

In [18]:
file_name = '../data/MYD09CMA.A2018015.006.2018017024441.hdf'
#file_lon = '../data/lat_MYD02HKM.output'
#file_lat = '../data/lon_MYD02HKM.output'

In [19]:
file = SD(file_name, SDC.READ)

In [20]:
print (file.info())

(17, 6)


In [21]:
datasets_dic = file.datasets()

for idx,sds in enumerate(datasets_dic.keys()):
    print (idx,sds)

0 Coarse Resolution Surface Reflectance Band 8
1 Coarse Resolution Surface Reflectance Band 9
2 Coarse Resolution Surface Reflectance Band 10
3 Coarse Resolution Surface Reflectance Band 11
4 Coarse Resolution Surface Reflectance Band 12
5 Coarse Resolution TOA Reflectance Band 1
6 Coarse Resolution TOA Reflectance Band 3
7 Coarse Resolution TOA Reflectance Band 8
8 Coarse Resolution TOA Reflectance Band 9
9 Coarse Resolution TOA Reflectance Band 10
10 Coarse Resolution AOT Model Residual Values
11 Coarse Resolution AOT at 550 nm
12 Coarse Resolution Atmospheric Optical Depth Model
13 Coarse Resolution Water Vapor
14 Coarse Resolution Air Temperature (2m)
15 Coarse Resolution Atmospheric Optical Depth QA
16 Coarse Resolution Number Mapping AOT


In [22]:
sds_obj = file.select('Coarse Resolution Atmospheric Optical Depth QA') # select sds

data = sds_obj.get() # get sds data
print (data)

[[127 127 127 ..., 127 127 127]
 [127 127 127 ..., 127 127 127]
 [127 127 127 ..., 127 127 127]
 ..., 
 [127  10  10 ...,  10  10  10]
 [127  10  10 ...,  10  10  10]
 [127  10  10 ...,  10  10  10]]


In [23]:
import pprint
pprint.pprint(sds_obj.attributes())

{'QA index': '\n'
             '\tNot bitmapped, but with the following values:\n'
             '\t0      inital value;\n'
             '\t1      None of the 500m pixels that correspond to this 1km '
             'pixel \n'
             '\t         has data that reaches either criteria for containing '
             'aerosols;\n'
             '\t2      Pixel is over water;\n'
             '\t3      One or more of the retrieved aerosol values has '
             'saturated\n'
             '\t         (and then all are set to a fill value);\n'
             '\t4      Pixel is cloudy or mixed or has a high band 26 value;\n'
             '\t5      Pixel had been flagged as water, but also had positive '
             'NDVI;\n'
             '\t6      Pixel is flagged as water and tests clear;\n'
             '\t7      Pixel is flagged as water and tests for turbidity;\n'
             '\t8      Snow (set in aot_filter());\n'
             '\t9      Bad geolocation, or high solar zenith;\n'
      

In [24]:
for key, value in sds_obj.attributes().items():
    print (key, value)
    if key == 'add_offset':
        add_offset = value  
    if key == 'scale_factor':
        scale_factor = value

_FillValue 127
valid_range [0, 19]
units (unitless)
long_name Coarse Resolution Atmospheric Optical Depth QA
QA index 
	Not bitmapped, but with the following values:
	0      inital value;
	1      None of the 500m pixels that correspond to this 1km pixel 
	         has data that reaches either criteria for containing aerosols;
	2      Pixel is over water;
	3      One or more of the retrieved aerosol values has saturated
	         (and then all are set to a fill value);
	4      Pixel is cloudy or mixed or has a high band 26 value;
	5      Pixel had been flagged as water, but also had positive NDVI;
	6      Pixel is flagged as water and tests clear;
	7      Pixel is flagged as water and tests for turbidity;
	8      Snow (set in aot_filter());
	9      Bad geolocation, or high solar zenith;
	10     Snow, or sunglint, or fire;
	11     Bad 500m data, no aerosol retrieval;
	12     Subpixel cloud (set in aot_filter());
	13     Aerosol retrieval anomalies;
	14     Possible salt-pan;
	15     Dese

In [25]:
data = (data - add_offset) * scale_factor
print (data)

[[ 12700.  12700.  12700. ...,  12700.  12700.  12700.]
 [ 12700.  12700.  12700. ...,  12700.  12700.  12700.]
 [ 12700.  12700.  12700. ...,  12700.  12700.  12700.]
 ..., 
 [ 12700.   1000.   1000. ...,   1000.   1000.   1000.]
 [ 12700.   1000.   1000. ...,   1000.   1000.   1000.]
 [ 12700.   1000.   1000. ...,   1000.   1000.   1000.]]


In [31]:
data.shape

(3600, 7200)

In [27]:
DATAFIELD_NAME = 'Coarse Resolution Atmospheric Optical Depth QA'

In [28]:
data3d = file.select(DATAFIELD_NAME)

In [29]:
data = data3d[11,:,:]

HDF4Error: ('get', 0, 'indexing expression exceeds variable number of dimensions')